In [1]:
import numpy as np
import pandas as pd
import os

new_dir = r"D:\ML\Vit-Yolo objD"
os.chdir(new_dir)
print("Current Directory:", os.getcwd())

Current Directory: D:\ML\Vit-Yolo objD


In [3]:
!pip install --upgrade ultralytics timm torch


   ---------------------------------------- 0.0/974.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/974.8 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/974.8 kB ? eta -:--:--
   -------------------------------- ------- 786.4/974.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 974.8/974.8 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.101
    Uninstalling ultralytics-8.3.101:
      Successfully uninstalled ultralytics-8.3.101


In [3]:
# hybrid_mobilevit_yolo.py
import torch
import timm
from torch import nn
from ultralytics import YOLO

# 1. MobileViT Feature Extractor
class MobileViTFeatureExtractor(nn.Module):
    def __init__(self, model_name='mobilevit_s'):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=True, features_only=True)
        self.proj = nn.Conv2d(self.backbone.feature_info[-1]['num_chs'], 256, kernel_size=1)

    def forward(self, x):
        features = self.backbone(x)[-1]  # use final stage
        return self.proj(features)

# 2. Hybrid Model with YOLO
class MobileViTYOLO(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.backbone = MobileViTFeatureExtractor()
        self.yolo = YOLO('yolov8n.yaml')
        self.yolo.model.model[0] = self.backbone  # Replace YOLO backbone

    def forward(self, x):
        return self.yolo(x)

# 3. Training Setup
def train():
    model = MobileViTYOLO()

    # Official YOLO training API
    model.yolo.train(
        data='your_dataset.yaml',
        epochs=100,
        imgsz=640,  # Match aspect ratio of your 1242x375 dataset
        rect=True,
        batch=8,
        optimizer='AdamW',
        lr0=1e-4
    )

if __name__ == "__main__":
    train()


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

Ultralytics 8.3.109  Python-3.12.7 torch-2.6.0+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=your_dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

train: Scanning D:\ML\Vit-Yolo objD\YOLO-dataset\labels\train.cache... 7481 images, 0 backgrounds, 0 corrupt: 100%|████

WARNING  'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False



val: Scanning D:\ML\Vit-Yolo objD\YOLO-dataset\labels\train.cache... 7481 images, 0 backgrounds, 0 corrupt: 100%|██████

Plotting labels to runs\detect\train3\labels.jpg... 


optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      4.001      4.847      4.295         43        640:   1%|          | 5/936 [00:04<13:24,  1.
Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x000001AE83BEB4C0>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\_weakrefset.py", line 40, in _remove
    self = selfref()
           ^^^^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 